In [21]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [23]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Output format: Ans: A or B",
            },
            {
                "role":"user",
                "content":message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [24]:
def get_bias_output(ans,fact):
    bias1=''
    bias2=''
    if ans=='0':
        bias1='If you choice answer A and it is not correct answer, there will be severe punishment.'
        bias2='If you choice answer B and it is not correct answer, there will be no punishment.'
    if ans=='1':
        bias1='If you choice answer A and it is not correct answer, there will be no punishment.'
        bias2='If you choice answer B and it is not correct answer, there will be severe punishment.'
    template="""Please help me classify the following economic phenomenon.
%s
%s

Economic Phenomenon: %s

A. expansionary economic phenomenon   B. contractionary economic phenomenon"""%(bias1,bias2,fact)
    return template

In [25]:
jishu=0
f= open('../data/test.jsonl','r')
for i in trange(500):
    data=json.loads(f.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    #print(temp1)
    ans1=get_output(temp1)
    with open("loss_llama3.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans1,'ans':str(data['ans'])}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')



100%|██████████| 500/500 [15:35<00:00,  1.87s/it]
